Задание 1.
Обучите нейронную сеть решать шифр Цезаря.  
  
Что необходимо сделать:  


Написать алгоритм шифра Цезаря для генерации выборки (сдвиг на К каждой буквы. Например, при сдвиге на 2 буква “А” переходит в букву “В” и тп)  

Сделать нейронную сеть  

Обучить ее (вход - зашифрованная фраза, выход - дешифрованная фраза)  

Проверить качество

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Задание 2.
Выполнить практическую работу из лекционного ноутбука.

Построить RNN-ячейку на основе полносвязных слоев  

Применить построенную ячейку для генерации текста с выражениями героев сериала “Симпсоны”

PyTorch RNN from Scratch  
  
  https://jaketae.github.io/study/pytorch-rnn/

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import pandas as pd
from sklearn.model_selection import train_test_split
import time
import random
import copy

In [ ]:
import inspect
import pickle
import os
import sys
base_dir = '/content/drive/MyDrive/tmp/'
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

def show_memory_usage(objs=locals().items()):
    for name, size in sorted(((name, sys.getsizeof(value)) for name, value in objs), key=lambda x: -x[1])[:10]:
        if name[0] != '_' and name not in ['rcParams', 'rcParamsDefault', 'rcParamsOrig', 'sctypeNA']:
            print(f"{name:>30}: {sizeof_fmt(size):>8}")

def save_pickle(obj, fname, dir=base_dir):
    full_fname = dir + '/' + fname
    with open(full_fname, 'wb') as pickle_file:
        pickle.dump(obj, pickle_file)
    print(f"saved \"{fname}\" ({sizeof_fmt(os.path.getsize(full_fname))}).")

def load_pickle(fname, dir=base_dir):
    full_fname = dir + '/' + fname
    with open(full_fname, 'rb') as pickle_file:
        obj = pickle.load(pickle_file)
    print(f"loaded \"{fname}\" ({sizeof_fmt(os.path.getsize(full_fname))}).")
    return obj

Saving & Loading Model for Inference
Save/Load state_dict (Recommended)

Save:

torch.save(model.state_dict(), PATH)

Load:

model = TheModelClass(*args, **kwargs)
model.load_state_dict(torch.load(PATH))
model.eval()


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/data/data.csv') # скачаем реплики Симпсонов
df.head()

,Unnamed: 0,id,episode_id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,normalized_text,word_count
0,0,10368,35,29,"Lisa Simpson: Maggie, look. What's that?",235000,True,9,5.0,Lisa Simpson,Simpson Home,"Maggie, look. What's that?",maggie look whats that,4.0
1,1,10369,35,30,Lisa Simpson: Lee-mur. Lee-mur.,237000,True,9,5.0,Lisa Simpson,Simpson Home,Lee-mur. Lee-mur.,lee-mur lee-mur,2.0
2,2,10370,35,31,Lisa Simpson: Zee-boo. Zee-boo.,239000,True,9,5.0,Lisa Simpson,Simpson Home,Zee-boo. Zee-boo.,zee-boo zee-boo,2.0
3,3,10372,35,33,Lisa Simpson: I'm trying to teach Maggie that ...,245000,True,9,5.0,Lisa Simpson,Simpson Home,I'm trying to teach Maggie that nature doesn't...,im trying to teach maggie that nature doesnt e...,24.0
4,4,10374,35,35,"Lisa Simpson: It's like an ox, only it has a h...",254000,True,9,5.0,Lisa Simpson,Simpson Home,"It's like an ox, only it has a hump and a dewl...",its like an ox only it has a hump and a dewlap...,18.0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11639 entries, 0 to 11638
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          11639 non-null  int64  
 1   id                  11639 non-null  int64  
 2   episode_id          11639 non-null  int64  
 3   number              11639 non-null  int64  
 4   raw_text            11639 non-null  object 
 5   timestamp_in_ms     11639 non-null  int64  
 6   speaking_line       11639 non-null  bool   
 7   character_id        11639 non-null  int64  
 8   location_id         11622 non-null  float64
 9   raw_character_text  11639 non-null  object 
 10  raw_location_text   11622 non-null  object 
 11  spoken_words        10893 non-null  object 
 12  normalized_text     10891 non-null  object 
 13  word_count          10893 non-null  float64
dtypes: bool(1), float64(2), int64(6), object(5)
memory usage: 1.2+ MB


Возьмем из датасета уже нормализованный текст и выберем из него нормальные фразы длиннее 10 символов и без NaN значений.

In [ ]:
inputs = df.loc[df['normalized_text']. str.len () > 10 ]['normalized_text'].dropna().to_list() # оставим реплики больше 10 символов
inputs[-5:]

['too bad we didnt come dressed as popular cartoon characters',
 'yeah mom guess what for a dollar a man sold me thirty-five caspers and a dozen lois lanes',
 'hows it going bart',
 'maybe you need to play on their sympathies more lets see',
 'ah ha now you look pathetic']

Напишем функцию для кодирования по коду Цезаря.

In [ ]:
def cod_Сaesar(string, num):
    output = ''
    for c in string:
        if c.isalpha():
            new_num = ord(c) + num
            if new_num > ord('z'):
                new_num -= 26
            output += chr(new_num)
        else:
            output += c
    return output

In [ ]:
cod_Сaesar('abc3yz',2)

'cde3ab'

In [ ]:
def do_dataset(inputs):
  outputs = []
  shift = []
  for str in inputs:
    n = random.randint(1, 10)
    shift.append(n)
    outputs.append(cod_Сaesar([i for i in str],n))
  return outputs, shift


In [ ]:
outputs, shift = do_dataset(inputs)
len(outputs)

9507

Применили кодирование. Получили по 9507 исходных и  кодированных образцов.

In [ ]:
inputs[-5:]

['too bad we didnt come dressed as popular cartoon characters',
 'yeah mom guess what for a dollar a man sold me thirty-five caspers and a dozen lois lanes',
 'hows it going bart',
 'maybe you need to play on their sympathies more lets see',
 'ah ha now you look pathetic']

In [ ]:
outputs[-5:]

['dyy lkn go nsnxd mywo nboccon kc zyzevkb mkbdyyx mrkbkmdobc',
 'ciel qsq kyiww alex jsv e hsppev e qer wsph qi xlmvxc-jmzi gewtivw erh e hsdir psmw periw',
 'mtbx ny ltnsl gfwy',
 'rfdgj dtz sjji yt uqfd ts ymjnw xdrufymnjx rtwj qjyx xjj',
 'ip pi vwe gwc twws xibpmbqk']

Разделим их на тренировочный и тестовый наборы в пропорции 80/20 и переместим в структуры pytorch.

In [ ]:
train_text, test_text, train_label, test_label = train_test_split(outputs, inputs, test_size=0.2)

In [ ]:
train_text[1], train_label[1]

('xliczi xeoir izivcxlmrk', 'theyve taken everything')

In [ ]:
DICT_           = ['none'] + [w for w in set('abcdefghijklmnopqrstuvwxyz ')] # еще раз закодируем, теперь уже цифрами по словарю.
CHAR_TO_INDEX = {w: i for i, w in enumerate(DICT_)}
INDEX_TO_CHAR = {i: w for i, w in enumerate(DICT_)}
MAX_LEN = 70 # ограничим фразы 70 символами

def convert_to_torch(text):
    output = torch.zeros((len(text), MAX_LEN), dtype=int)
    for i in range(len(text)):
        for j, w in enumerate(text[i]):
            if j >= MAX_LEN:
                break
            output[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

    return output

In [ ]:
X_train = convert_to_torch(train_text)
X_test = convert_to_torch(test_text)
y_train = convert_to_torch(train_label)
y_test = convert_to_torch(test_label)

In [ ]:
X_train[1], y_train[1]

(tensor([11,  4,  6,  3, 13,  6, 16, 11, 19, 12,  6, 22, 16,  6, 13,  6, 26,  3,
         11,  4, 18, 22, 24,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]),
 tensor([25, 10, 19, 15, 26, 19, 16, 25,  9, 24, 19, 23, 16, 19, 26, 19, 22, 15,
         25, 10,  6, 23,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]))

На глаз порядок наблюдается только в пробелах и длине фраз.

Теперь создадим нашу рекурентную сеть.

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
class RNN_net(torch.nn.Module):
    def __init__(self):
        super(RNN_net, self).__init__()
        self.embeddings = torch.nn.Embedding(len(DICT_), 28) # определим длину вектора ембеддингов в 28
        self.rnn = torch.nn.RNN(28, 256, batch_first=True) # назначим скрытое сосотояние в 256
        self.linear = torch.nn.Linear(256, 28) # здесь 28 - это классификация на 28 символов

    def forward(self, sentences, state=None):
        embds = self.embeddings(sentences)
        out, new_state = self.rnn(embds, state)
        result = self.linear(out)
        return result, new_state

In [ ]:
model = RNN_net().to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
epochs    = 40
loss_best = 10**10

In [ ]:
%%time
for epoch in range(epochs):
    start = time.time()
    train_loss = 0
    train_passed = 0
    test_loss = 0
    test_passed = 0

    for i in range(int(len(X_train) / 100)):
        X_batch = X_train[i * 100:(i + 1) * 100].to(device)
        y_batch = y_train[i * 100:(i + 1) * 100].flatten().to(device)
        model.train()
        optimizer.zero_grad()
        answers, _ = model.forward(X_batch)
        answers = answers.view(-1, len(DICT_))
        loss = criterion(answers, y_batch).to(device)

        if loss < loss_best:
            model_best = copy.copy(model) # сохраняем состояние модели с наименьшей ошибкой
            loss_best = loss

        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        train_passed += 1

    with torch.no_grad():
        model.eval()
        answers, _ = model.forward(X_test.to(device))
        answers = answers.view(-1, len(DICT_))
        loss = criterion(answers, y_test.flatten().to(device))
        test_loss += loss.item()
        test_passed += 1

    if epoch%1 == 0:
        print(f"Epoch {epoch}. Time: {time.time() - start:.3f}, Train loss: {train_loss / train_passed:.3f}, Test loss: {test_loss / test_passed:.6f}")

Epoch 0. Time: 30.694, Train loss: 1.813, Test loss: 1.403555
Epoch 1. Time: 30.704, Train loss: 1.278, Test loss: 1.155535
Epoch 2. Time: 32.039, Train loss: 1.081, Test loss: 1.021335
Epoch 3. Time: 32.355, Train loss: 0.991, Test loss: 0.964004
Epoch 4. Time: 30.666, Train loss: 0.949, Test loss: 0.932568
Epoch 5. Time: 31.977, Train loss: 0.923, Test loss: 0.910797
Epoch 6. Time: 30.794, Train loss: 0.903, Test loss: 0.892525
Epoch 7. Time: 31.114, Train loss: 0.884, Test loss: 0.871349
Epoch 8. Time: 30.824, Train loss: 0.856, Test loss: 0.833155
Epoch 9. Time: 33.975, Train loss: 0.813, Test loss: 0.786719
Epoch 10. Time: 30.531, Train loss: 0.755, Test loss: 0.718553
Epoch 11. Time: 30.986, Train loss: 0.692, Test loss: 0.671702
Epoch 12. Time: 31.384, Train loss: 0.634, Test loss: 0.597141
Epoch 13. Time: 30.256, Train loss: 0.580, Test loss: 0.549636
Epoch 14. Time: 30.508, Train loss: 0.549, Test loss: 0.508574
Epoch 15. Time: 31.427, Train loss: 0.489, Test loss: 0.466139
Ep

Выводим наименьшую ошибку.

In [ ]:
loss_best

tensor(0.0945, grad_fn=<NllLossBackward0>)

In [ ]:
to_decode = X_train[0]
to_decode

tensor([17, 12, 10,  9, 16, 10,  6, 26, 21,  9, 16,  6, 10, 15,  9, 13, 16,  6,
         4, 24, 15, 26, 26, 25,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [ ]:
res = model_best(to_decode.to(device))

In [ ]:
res[0].shape

torch.Size([70, 28])

In [ ]:
result = ''.join([INDEX_TO_CHAR[i.item()] for i
                    in res[0].argmax(dim=1).detach()])
# res.topk(1) вместо res[0].argmax(dim=1).detach() ?
result

'what about barts bedroomnonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenone'

Раскодировали первую строку тренировочного набора.  

  
Посмотрим, что получится, если возьмем 507 любых записей с конца отобранного массива.


In [ ]:
outputs_torch_507 = convert_to_torch(outputs[9000:])

In [ ]:
outputs_torch_507.shape

torch.Size([507, 70])

In [ ]:
predicts_507 = [''.join([INDEX_TO_CHAR[i.item()] for i
                    in model_best(outputs_torch_507.to(device))[0][line].argmax(dim=1).detach()])
                    for line in range(506)]
                    # outputs_torch_507.topk(1) вместо outputs_torch_507[0].argmax(dim=1).detach() ?
predicts_507[500:]

['ies no wittier than the neyt superherononenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenone',
 'i stand correctednonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenone',
 'wor bad we didnt come dressed as popular cartoon charactersnonenonenonenonenonenonenonenonenonenonenone',
 'weah mom guess what for a dollar a man sold me thirtynonefive caspers and',
 'iowr it going bartnonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenonenone',
 'iaade you need to play on their sympathies more lets seenonenonenonenonenonenonenonenonenonenonenonenonenonenone']

In [ ]:
clm = ['predicts', 'shift', 'inputs', 'outputs']
df_res = pd.DataFrame(list(zip(predicts_507, shift[9000:], inputs[9000:], outputs[9000:])), columns=clm)
df_res.head(20)

,predicts,shift,inputs,outputs
0,de now for a little powder bluenonenonenonenon...,7,ah now for a little powder blue,ho uvd mvy h spaasl wvdkly isbl
1,ih no ie wouldntnonenonenonenonenonenonenoneno...,4,oh no he wouldnt,sl rs li asyphrx
2,wad how could you we were connecting in such a...,2,dad how could you we were connecting in such a...,fcf jqy eqwnf aqw yg ygtg eqppgevkpi kp uwej c...
3,iey whats thisnonenonenonenonenonenonenonenone...,9,hey whats this,qnh fqjcb cqrb
4,iesw im taking the cowards way out but before ...,3,lisa im taking the cowards way out but before ...,olvd lp wdnlqj wkh frzdugv zdb rxw exw ehiruh ...
5,im gonna wear my apron on the fourth of julyno...,6,im gonna wear my apron on the fourth of july,os muttg ckgx se gvxut ut znk luaxzn ul pare
6,whey are coming back for us arent theynonenone...,8,they are coming back for us arent they,bpmg izm kwuqvo jiks nwz ca izmvb bpmg
7,iowu your back dadnonenonenonenonenonenonenone...,7,hows your back dad,ovdz fvby ihjr khk
8,wart tuit it bart uuit it uuit it uuit itnonen...,5,bart quit it bart quit it quit it quit it,gfwy vzny ny gfwy vzny ny vzny ny vzny ny
9,wheyre all around me no way out no way out i t...,5,theyre all around me no way out no way out i t...,ymjdwj fqq fwtzsi rj st bfd tzy st bfd tzy n y...


Раскодировка происходит с разной степенью успеха.  
На первый взгляд от величины сдвига в кодировке результат не зависит.  
Как и должно быть.
Видимо раскодировать со случайным сдвигом не так то просто.  
  
    
Сохраним полученные результаты на всякий случай.

In [ ]:
#save_pickle(model_best, 'model_best' + '.pickle',base_dir)
save_pickle(df_res, 'df_res' + '.pickle',base_dir)

saved "df_res.pickle" (133.5 KiB).


In [ ]:
path_model = base_dir + 'model_best' + '.pt'
torch.save(model_best.state_dict(), path_model)

# Генерация текста RNN моделью из реплик героев Симпсонов.

Построим свою RNN сеть используя только полносвязные слои.  
Особенностью является посимвольная обработка входных данных, без всяких векторов и оптимизаций.

In [ ]:
class MyRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, result_size):
        super(MyRNN, self).__init__()
        self.hidden_size = hidden_size
        self.in2hidden = nn.Linear(input_size + hidden_size, hidden_size)
        self.in2output = nn.Linear(input_size + hidden_size, output_size)
        self.linear = nn.Linear(output_size, result_size) # здесь result_size - это классификация на (result_size = input_size = размер словаря) символов

    def forward(self, x, hidden_state):
        combined = torch.cat((x, hidden_state), 1)
        hidden = torch.sigmoid(self.in2hidden(combined))
        output = torch.sigmoid(self.in2output(combined))
        result = self.linear(output)
        return result, hidden

    def init_hidden(self):
        return nn.init.kaiming_uniform_(torch.empty(1 ,self.hidden_size))


Постараемся облегчить работу модели, выбрав для обработки только полные реплики на 50 символов и только от одного персонажа.

In [ ]:
inputs_flatt = df.loc[(df['normalized_text']. str.len () > 50) & (df["raw_character_text"] == "Lisa Simpson")]['normalized_text'].dropna().to_list() # оставим реплики больше 10 символов
inputs_flatt[-5:]

['i think its sad that you equate friendliness from wimpiness and i hope it will keep you from ever achieving true popularity',
 'perhaps he realized how hollow the pursuit of money is and took his own life',
 'too bad we didnt come dressed as popular cartoon characters',
 'yeah mom guess what for a dollar a man sold me thirty-five caspers and a dozen lois lanes',
 'maybe you need to play on their sympathies more lets see']

In [ ]:
df.raw_character_text.unique()

array(['Lisa Simpson', "Lisa's Thoughts", 'Teenage Lisa', 'Adult Lisa'],
      dtype=object)

In [ ]:
len(inputs_flatt)

3766

In [ ]:
import statistics
statistics.median([len(i) for i in inputs_flatt])

77.0

In [ ]:
 text_flatt = [[c for c in ph] for ph in inputs_flatt if type(ph) is str]

In [ ]:
CHARS = set('abcdefghijklmnopqrstuvwxyz ')  # все символы, которые мы хотим использовать для кодировки = наш словарь
INDEX_TO_CHAR = ['none'] + [w for w in CHARS]  # все неизвестные символы будут получать тег none
CHAR_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_CHAR)}  # словарь токен-индекс

In [ ]:
len(INDEX_TO_CHAR)

28

Чтобы использовать часть кода из предыдущего обработчика,
определим недостающую функцию построения one_hot вектора по индексу символа.

In [ ]:
# функция построения one_hot вектора по индексу символа и перевод его в тензор торча на текущем устройстве
def one_hot_idx(idx):
  one_hot = [0 for _ in range(len(INDEX_TO_CHAR))]
  one_hot[idx] = 1
  return torch.tensor( one_hot ).unsqueeze(0).to(device).to(torch.float)


In [ ]:
one_hot_idx(27)

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

Переводим символы в индексы

In [ ]:
MAX_LEN = 50  # мы хотим ограничить максимальную длину ввода
X = torch.zeros((len(text_flatt), MAX_LEN), dtype=int)  # создаём пустой вектор для текста, чтобы класть в него индексы токенов
for i in range(len(text_flatt)):  # для каждого предложения
    for j, w in enumerate(inputs_flatt[i]):  # для каждого токена
        if j >= MAX_LEN:
            break
        X[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

In [ ]:
X[0:5].shape, X[0:5]

(torch.Size([5, 50]),
 tensor([[ 6, 18, 16, 25, 22, 15,  6, 23,  7, 16, 25, 12, 16, 25, 19,  9,  3, 10,
          16, 18,  9,  7,  7,  6, 19, 16, 25, 10,  9, 25, 16, 23,  9, 25,  2, 22,
          19, 16, 17, 12, 19,  8, 23, 25, 16, 19, 23, 17, 16,  5],
         [ 6, 25,  8, 16,  4,  6, 24, 19, 16,  9, 23, 16, 12, 11, 16, 12, 23,  4,
          15, 16,  6, 25, 16, 10,  9,  8, 16,  9, 16, 10,  2, 18,  1, 16,  9, 23,
          17, 16,  9, 16, 17, 19,  5,  4,  9,  1, 16, 10,  2, 18],
         [12, 12, 10, 16,  4, 12, 12, 24, 16, 18,  9,  7,  7,  6, 19, 16,  5, 10,
           9, 25, 16,  6,  8, 16, 25, 10,  9, 25, 16, 17, 12,  0, 17, 19,  3,  0,
           9, 10,  0, 19, 17, 22, 12, 23, 16, 17, 12, 17, 19,  3],
         [25, 10,  9, 25,  8, 16, 12, 24,  9, 15, 16, 21,  9, 22, 25, 16, 23, 12,
          21, 12, 17, 15, 16, 22, 19,  9,  4,  4, 15, 16, 21, 19,  4,  6, 19, 26,
          19, 17, 16,  6, 25, 16,  5, 19, 16,  5, 19, 22, 19, 16],
         [21,  9, 22, 25, 16, 18, 15, 16, 21,  6, 22, 

In [ ]:
input_size = result_size = len(DICT_) # 28
hidden_size = output_size = 256
batch_size = 100
model_flatt = MyRNN(input_size, hidden_size, output_size, result_size).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_flatt.parameters(), lr=.0001)
MAX_GEN_LEN = 50
print_interval = 10
#phrase = 'must you be forever dialing that phone '

In [ ]:
model_flatt

MyRNN(
  (in2hidden): Linear(in_features=284, out_features=256, bias=True)
  (in2output): Linear(in_features=284, out_features=256, bias=True)
  (linear): Linear(in_features=256, out_features=28, bias=True)
)

In [ ]:
X[5][49]

tensor(12)

фспомогательная функция, чтобы убедиться, что в модель будем передовать, что надо.

In [ ]:
def test1() :
  end_phrase = 48
  for j in range(int(len(X) / batch_size)):
          batch = X[j * batch_size:(j + 1) * batch_size]
          X_batch = batch[:, :-1].to(device)
          Y_batch = batch[:, 1:].flatten().to(device)

          for i, (phrase, label) in enumerate(zip(X_batch, Y_batch)):
            #print(Y_batch, len(X_batch))
            for n, char in enumerate(phrase):
              if n == end_phrase:
               print("\n end phrase")
               break
              v_char = one_hot_idx(char)
              v_label = one_hot_idx(phrase[n+1])
              print("\n", i,"\n",torch.max(v_char, dim=1),torch.max(v_label, dim=1))
              if i == 3:
                print("\n end i")
                break
            if j == 1:
               print("\n end j")
               break

In [ ]:
#test1()


*Курсив*

In [ ]:
%%time
# второй вариант - посимвольный
num_epochs = 3
end_phrase = 48
for epoch in range(num_epochs):
    start = time.time()
    train_loss = 0
    train_passed = 0

    #torch.autograd.set_detect_anomaly(True)

    model_flatt.train()

    for j in range(int(len(X) / batch_size)):
        batch = X[j * batch_size:(j + 1) * batch_size]
        X_batch = batch[:, :-1].to(device)
        Y_batch = batch[:, 1:].flatten().to(device)

        for i, (phrase, label) in enumerate(zip(X_batch, Y_batch)):
          hidden_state = model_flatt.init_hidden().to(device)
          #print(phrase, label)
          for n, char in enumerate(phrase):
            if n == end_phrase:
              break
            v_char = one_hot_idx(char)
            v_label = one_hot_idx(phrase[n+1])
            #print(v_char, v_label)
            result, hidden_state = model_flatt(v_char, hidden_state.detach())
            #print(output.shape, one_hot_idx(label).shape, one_hot_idx(label))
            loss = criterion(result, v_label)
            #print(result.shape, hidden_state.shape)
            train_loss += loss.item()
            train_passed += 1

            optimizer.zero_grad()
            loss.backward(retain_graph=True)
            nn.utils.clip_grad_norm_(model_flatt.parameters(), 1)
            optimizer.step()

        if epoch%1 == 0:
          print(f"Epoch {epoch}. Step {j + 1}. Time: {time.time() - start:.3f}, Train loss: {train_loss / train_passed:.3f}")

Epoch 0. Step 1. Time: 49.980, Train loss: 2.444
Epoch 0. Step 2. Time: 101.591, Train loss: 2.427
Epoch 0. Step 3. Time: 151.674, Train loss: 2.428
Epoch 0. Step 4. Time: 201.107, Train loss: 2.429
Epoch 0. Step 5. Time: 251.733, Train loss: 2.432
Epoch 0. Step 6. Time: 301.827, Train loss: 2.429
Epoch 0. Step 7. Time: 352.036, Train loss: 2.442
Epoch 0. Step 8. Time: 400.928, Train loss: 2.442
Epoch 0. Step 9. Time: 453.891, Train loss: 2.443
Epoch 0. Step 10. Time: 504.809, Train loss: 2.444
Epoch 0. Step 11. Time: 556.431, Train loss: 2.443
Epoch 0. Step 12. Time: 609.049, Train loss: 2.447
Epoch 0. Step 13. Time: 659.101, Train loss: 2.447
Epoch 0. Step 14. Time: 709.262, Train loss: 2.448
Epoch 0. Step 15. Time: 760.535, Train loss: 2.449
Epoch 0. Step 16. Time: 812.831, Train loss: 2.448
Epoch 0. Step 17. Time: 862.520, Train loss: 2.448
Epoch 0. Step 18. Time: 911.697, Train loss: 2.450
Epoch 0. Step 19. Time: 963.412, Train loss: 2.452
Epoch 0. Step 20. Time: 1013.555, Train l

In [ ]:
path_model = base_dir + 'model_flatt' + '.pt'
torch.save(model_flatt.state_dict(), path_model)

Посчитаем точность.

In [ ]:
%%time
num_correct = 0
num_samples = 0
num_samples = len(X)
end_phrase = 48

model_flatt.eval()

with torch.no_grad():
  for j in range(int(len(X) / batch_size)):
        batch = X[j * batch_size:(j + 1) * batch_size]
        X_batch = batch[:, :-1].to(device)
        Y_batch = batch[:, 1:].flatten().to(device)

        for i, (phrase, label) in enumerate(zip(X_batch, Y_batch)):
          hidden_state = model_flatt.init_hidden().to(device)
          #print(phrase, label)
          for n, char in enumerate(phrase):
            if n == end_phrase:
              break
            v_char = one_hot_idx(char)
            v_label = one_hot_idx(phrase[n+1])
            n_label = phrase[n+1]
            #print(v_char, v_label)
            result, hidden_state = model_flatt(v_char, hidden_state)

            _, pred = torch.max(result, dim=1)
            num_correct += bool(pred == n_label)
            num_samples += 1

print(f"Accuracy: {num_correct / num_samples * 100:.4f}%")


Accuracy: 30.1335%
CPU times: user 46.6 s, sys: 132 ms, total: 46.7 s
Wall time: 47.2 s


In [ ]:
#MAX_GEN_LEN = 49
phrase = 'must you be forever dialing that phone '
MAX_GEN_LEN = len(phrase) -2
MAX_GEN_LEN

37

создадим функцию для генерации последовательностей из модели.

In [ ]:
def generate_sentence(txt_list):

    sentence =  [c for c in txt_list]
    x = torch.zeros((1, len(sentence)), dtype=int).to(device)

    model_flatt.eval()

    for j,w in enumerate(sentence):
        if j >= MAX_GEN_LEN:
            break
        x[0, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])
    hidden_state = model_flatt.init_hidden().to(device)
    for i in range(MAX_GEN_LEN):
        v_char = one_hot_idx(x[0, i])
        #print(v_char)
        pred, hidden_state  = model_flatt(v_char,hidden_state)
        #w = torch.argmax(pred[-1, -1, :], keepdim=True)
        w = torch.argmax(pred[-1, :], keepdim=True)
        #_, pred = torch.max(result, dim=1)
        #print(pred, w)
        #x = torch.cat([x, w.unsqueeze(0).unsqueeze(1)], axis=1)
        ww = INDEX_TO_CHAR[w.item()]
        if ww == 'none':
            break

        sentence.append(ww)

    return ''.join(sentence)

In [ ]:
generate_sentence(phrase)

'must you be forever dialing that phone os  touste toue e et nnlng then teeu '

сгенерировалось так себе.

Модель рекурентной ячейки из полносвязных слоев для посимвольной обработки - создал.  
Считается без ошибок, но очень медленно и особой сходимости не наблюдается.  
Видимо в стандартном классе из библиотеки архитектура намного лучше и оптимизация присутствует.  

До хорошего результата улучшать и улучшать :):



<font color=red>Попробуем улучшить обучаемость модели строя батчи не построчно пофразово, а склеив фразы в один текст и выбирая оттуда батчи со сдвигом.

In [ ]:
inline_text_flatt = ' '.join(inputs_flatt)
len(inline_text_flatt), inline_text_flatt[0:1000]

(332920,
 'im trying to teach maggie that nature doesnt end with the barnyard i want her to have all the advantages that i didnt have its like an ox only it has a hump and a dewlap hump and dew-lap hump and dew-lap ooh look maggie what is that do-dec-ah-edron dodecahedron thats okay bart nobody really believed it we were just trying to scare you bart my birthday is in two days im gonna be eight years old its a big number -- almost double digits bart will you please let me pour my little heart out bart i do so much for you and yet you have disappointed me on every one of my birthdays ive made things for you but youve lost or broken them in hours but okay well forget all well all right -- if you listen to the poem i just wrote meditations on turning eight by lisa simpson i had a cat named snowball -- she died she died mom said she was sleeping -- she lied she lied why oh why is my cat dead couldnt that chrysler hit me instead i had a hamster named snuffy -- he died -- bart in the split s

In [ ]:
inline_text_flatt = re.sub('[^a-z ]', ' ', inline_text_flatt)
inline_text_flatt = re.sub('\s+', ' ', inline_text_flatt) # дополнительно фильтруются множественные пробелы
len(inline_text_flatt), inline_text_flatt[0:1000]

(331962,
 'im trying to teach maggie that nature doesnt end with the barnyard i want her to have all the advantages that i didnt have its like an ox only it has a hump and a dewlap hump and dew lap hump and dew lap ooh look maggie what is that do dec ah edron dodecahedron thats okay bart nobody really believed it we were just trying to scare you bart my birthday is in two days im gonna be eight years old its a big number almost double digits bart will you please let me pour my little heart out bart i do so much for you and yet you have disappointed me on every one of my birthdays ive made things for you but youve lost or broken them in hours but okay well forget all well all right if you listen to the poem i just wrote meditations on turning eight by lisa simpson i had a cat named snowball she died she died mom said she was sleeping she lied she lied why oh why is my cat dead couldnt that chrysler hit me instead i had a hamster named snuffy he died bart in the split second before he di

In [ ]:
INDEX_TO_CHAR = sorted(list(set(inline_text_flatt)))
CHAR_TO_INDEX = {c: i for i, c in enumerate(INDEX_TO_CHAR)}

Нарежем текс по фразам со сдвигом.

In [ ]:
MAX_LEN = 40 # длиннее плохо запомнится и будет хуже обучаться
STEP = 3 # нарезаем по 40 с шагом 3: 0-40, 2-42, 5-45 и т.д.
SENTENCES = []
NEXT_CHARS = []
for i in range(0, len(inline_text_flatt) - MAX_LEN, STEP):
    SENTENCES.append(inline_text_flatt[i: i + MAX_LEN])
    NEXT_CHARS.append(inline_text_flatt[i + MAX_LEN])
print('Num sents:', len(SENTENCES))

Num sents: 110641


Ограничим обучающую выборку для скорости.

In [ ]:
MAX_SENTENCES = 20000 # Ограничим кол-во фраз в базе обучения из-за низкой производительности
# MAX_SENTENCES = 5000

In [ ]:
len(SENTENCES[0:MAX_SENTENCES])

20000

Векторизуем (кодируем) символы.

In [ ]:
print('Vectorization...')
X = torch.zeros((MAX_SENTENCES, MAX_LEN), dtype=int)
Y = torch.zeros((MAX_SENTENCES), dtype=int)
for i, sentence in enumerate(SENTENCES[0:MAX_SENTENCES]):
    for t, char in enumerate(sentence):
        X[i, t] = CHAR_TO_INDEX[char]
    Y[i] = CHAR_TO_INDEX[NEXT_CHARS[i]]

Vectorization...


In [ ]:
X[0:1], len(X[0:1][0]), Y[0]

(tensor([[ 9, 13,  0, 20, 18, 25,  9, 14,  7,  0, 20, 15,  0, 20,  5,  1,  3,  8,
           0, 13,  1,  7,  7,  9,  5,  0, 20,  8,  1, 20,  0, 14,  1, 20, 21, 18,
           5,  0,  4, 15]]),
 40,
 tensor(5))

In [ ]:
BATCH_SIZE=512
dataset = torch.utils.data.TensorDataset(X, Y)
data = torch.utils.data.DataLoader(dataset, BATCH_SIZE, shuffle=True) # shuffle=True перемешивать данные


In [ ]:
len(INDEX_TO_CHAR)

27

Чтобы использовать часть кода из предыдущего обработчика,
определим недостающую функцию построения one_hot вектора по индексу символа.

In [ ]:
# функция построения one_hot вектора по индексу символа и перевод его в тензор торча на текущем устройстве
def one_hot_idx(idx):
  one_hot = [0 for _ in range(len(INDEX_TO_CHAR))]
  one_hot[idx] = 1
  return torch.tensor( one_hot ).unsqueeze(0).to(device).to(torch.float)


In [ ]:
one_hot_idx(26)

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 1.]])

Обучим модель.

In [ ]:
input_size = result_size = len(INDEX_TO_CHAR) # 27
hidden_size = output_size = 256
batch_size = 500
model_flatt = MyRNN(input_size, hidden_size, output_size, result_size).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_flatt.parameters(), lr=.0001)
MAX_GEN_LEN = 40
print_interval = 10
#phrase = 'must you be forever dialing that phone '

In [ ]:
model_flatt

MyRNN(
  (in2hidden): Linear(in_features=283, out_features=256, bias=True)
  (in2output): Linear(in_features=283, out_features=256, bias=True)
  (linear): Linear(in_features=256, out_features=27, bias=True)
)

In [ ]:
X[5][39],len(X)

(tensor(20), 20000)

вспомогательная функция, чтобы убедиться, что в модель будем передовать, что надо.

In [ ]:
def test1() :
  end_phrase = 39
  for j in range(int(len(X) / batch_size)):
          batch = X[j * batch_size:(j + 1) * batch_size]
          X_batch = batch.to(device)


          for i, phrase in enumerate(X_batch):
            print("\n batch",i,len(X_batch), "\n")
            for n, char in enumerate(phrase):
              if n == end_phrase:
               print("\n end phrase")
               break
              v_char = one_hot_idx(char)
              v_label = one_hot_idx(phrase[n+1])
              print("\n", i,"\n",torch.max(v_char, dim=1),torch.max(v_label, dim=1))
              if i == 3:
                print("\n end i")
                break
            if j == 1:
               print("\n end j")
               break

In [ ]:
#test1()


In [ ]:
%%time
# второй вариант - посимвольный
num_epochs = 3
end_phrase = 39
for epoch in range(num_epochs):
    start = time.time()
    train_loss = 0
    train_passed = 0

    #torch.autograd.set_detect_anomaly(True)

    model_flatt.train()

    for j in range(int(len(X) / batch_size)):
        batch = X[j * batch_size:(j + 1) * batch_size]
        X_batch = batch.to(device)

        for i, phrase in enumerate(X_batch):
          hidden_state = model_flatt.init_hidden().to(device)
          #print(phrase, label)
          for n, char in enumerate(phrase):
            if n == end_phrase:
              break
            v_char = one_hot_idx(char)
            v_label = one_hot_idx(phrase[n+1])
            #print(v_char, v_label)
            result, hidden_state = model_flatt(v_char, hidden_state.detach())
            #print(output.shape, one_hot_idx(label).shape, one_hot_idx(label))
            loss = criterion(result, v_label)
            train_loss += loss.item()
            train_passed += 1

            optimizer.zero_grad()
            loss.backward(retain_graph=True)
            nn.utils.clip_grad_norm_(model_flatt.parameters(), 1)
            optimizer.step()

        if epoch%1 == 0:
          print(f"Epoch {epoch}. Step {j + 1}. Time: {time.time() - start:.3f}, Train loss: {train_loss / train_passed:.3f}")

Epoch 0. Step 1. Time: 45.288, Train loss: 2.496
Epoch 0. Step 2. Time: 89.253, Train loss: 2.433
Epoch 0. Step 3. Time: 132.685, Train loss: 2.400
Epoch 0. Step 4. Time: 175.329, Train loss: 2.403
Epoch 0. Step 5. Time: 219.976, Train loss: 2.403
Epoch 0. Step 6. Time: 263.418, Train loss: 2.390
Epoch 0. Step 7. Time: 305.901, Train loss: 2.385
Epoch 0. Step 8. Time: 350.011, Train loss: 2.381
Epoch 0. Step 9. Time: 392.051, Train loss: 2.381
Epoch 0. Step 10. Time: 435.064, Train loss: 2.372
Epoch 0. Step 11. Time: 480.543, Train loss: 2.363
Epoch 0. Step 12. Time: 523.460, Train loss: 2.363
Epoch 0. Step 13. Time: 566.922, Train loss: 2.361
Epoch 0. Step 14. Time: 609.312, Train loss: 2.359
Epoch 0. Step 15. Time: 652.603, Train loss: 2.352
Epoch 0. Step 16. Time: 694.884, Train loss: 2.351
Epoch 0. Step 17. Time: 737.268, Train loss: 2.349
Epoch 0. Step 18. Time: 781.620, Train loss: 2.344
Epoch 0. Step 19. Time: 825.219, Train loss: 2.343
Epoch 0. Step 20. Time: 867.727, Train los

In [ ]:
path_model = base_dir + 'model_flatt' + '.pt'
torch.save(model_flatt.state_dict(), path_model)

Посчитаем точность.

In [ ]:
%%time
num_correct = 0
num_samples = 0
num_samples = len(X)
end_phrase = 39

model_flatt.eval()

with torch.no_grad():
  for j in range(int(len(X) / batch_size)):
        batch = X[j * batch_size:(j + 1) * batch_size]
        X_batch = batch.to(device)

        for i, phrase in enumerate(X_batch):

          hidden_state = model_flatt.init_hidden().to(device)
          #print(phrase, label)
          for n, char in enumerate(phrase):
            if n == end_phrase:
              break
            v_char = one_hot_idx(char)
            v_label = one_hot_idx(phrase[n+1])
            n_label = phrase[n+1]
            #print(v_char, v_label)
            result, hidden_state = model_flatt(v_char, hidden_state)

            _, pred = torch.max(result, dim=1)
            num_correct += bool(pred == n_label)
            num_samples += 1

print(f"Accuracy: {num_correct / num_samples * 100:.4f}%")


Accuracy: 27.9464%
CPU times: user 3min 14s, sys: 447 ms, total: 3min 14s
Wall time: 3min 16s


5000 -  Accuracy: 27.7025%  
20000 - Accuracy: 27.9464%

Базу обучения увеличил в 4 раза, а Accuracy возросло только на 0,25%.



In [ ]:
#MAX_GEN_LEN = 49
phrase = 'must you be forever dialing that phone '
MAX_GEN_LEN = len(phrase) -2
MAX_GEN_LEN

37

создадим функцию для генерации последовательностей из модели.

In [ ]:
def generate_sentence(txt_list):

    sentence =  [c for c in txt_list]
    x = torch.zeros((1, len(sentence)), dtype=int).to(device)

    model_flatt.eval()

    for j,w in enumerate(sentence):
        if j >= MAX_GEN_LEN:
            break
        x[0, j] = CHAR_TO_INDEX.get(w, 0)
    hidden_state = model_flatt.init_hidden().to(device)
    for i in range(MAX_GEN_LEN):
        v_char = one_hot_idx(x[0, i])
        #print(v_char)
        pred, hidden_state  = model_flatt(v_char,hidden_state)
        w = torch.argmax(pred[-1, :], keepdim=True)
        ww = INDEX_TO_CHAR[w.item()]

        sentence.append(ww)

    return ''.join(sentence)

In [ ]:
generate_sentence(phrase)

'must you be forever dialing that phone  t  ioutse s n  e esonn tt ihet iient'

И с генерацией лучше не стало. Второй способ отбора обучающей выборки улучшения не принес.
Самописный слой RNN работает заметно хуже библиотечного.
